In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

(X_train, y_train), (X_test, y_test)=imdb.load_data(num_words=5000)
X_train=sequence.pad_sequences(X_train, maxlen=500)
X_test=sequence.pad_sequences(X_test, maxlen=500)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
D:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
D:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must sp

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

embedding_vector_length=32
model=Sequential()
model.add(Embedding(5000, embedding_vector_length, input_length=500))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping=EarlyStopping(patience=2)
hist=model.fit(X_train, y_train, epochs=10, validation_split=0.3, batch_size=64)

Epoch 1/10
274/274 [==============================] - 138s 496ms/step - loss: 0.6039 - accuracy: 0.6476 - val_loss: 0.3439 - val_accuracy: 0.8531
Epoch 2/10
274/274 [==============================] - 137s 500ms/step - loss: 0.3085 - accuracy: 0.8735 - val_loss: 0.3454 - val_accuracy: 0.8499
Epoch 3/10
274/274 [==============================] - 139s 507ms/step - loss: 0.2536 - accuracy: 0.8998 - val_loss: 0.3207 - val_accuracy: 0.8688
Epoch 4/10
274/274 [==============================] - 139s 509ms/step - loss: 0.2567 - accuracy: 0.8997 - val_loss: 0.3577 - val_accuracy: 0.8455
Epoch 5/10
274/274 [==============================] - 139s 508ms/step - loss: 0.2042 - accuracy: 0.9229 - val_loss: 0.3513 - val_accuracy: 0.8729
Epoch 6/10
274/274 [==============================] - 135s 493ms/step - loss: 0.1538 - accuracy: 0.9444 - val_loss: 0.3791 - val_accuracy: 0.8561
Epoch 7/10
274/274 [==============================] - 135s 494ms/step - loss: 0.1444 - accuracy: 0.9492 - val_loss: 0.3739 -

In [6]:
scores=model.evaluate(X_train, y_train, verbose=1)
print(f'Accuracy : {scores[1]*100:.2f}')
scores=model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy : {scores[1]*100:.2f}')

782/782 [==============================] - 37s 47ms/step - loss: 0.1844 - accuracy: 0.94263s - loss: 0.155 - ETA: 2s - loss: 0.1616 - 
Accuracy : 94.26
782/782 [==============================] - 38s 49ms/step - loss: 0.4518 - accuracy: 0.8564
Accuracy : 85.64


In [8]:
%matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.plot(hist.history['loss'])
plt.plot(hist.history['accuracy'])
plt.legend(['loss','accuracy'])
plt.grid()
plt.show()

Using matplotlib backend: Qt5Agg


In [9]:
import pandas as pd

df=pd.read_csv("d:/data/customer/customer.csv")
df.head()

,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [12]:
train_cols=df.columns[:-1]
X=df[train_cols]
y=df["Churn"]

In [13]:
from imblearn.under_sampling import RandomUnderSampler

X_sample, y_sample=RandomUnderSampler(random_state=0).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample, columns=train_cols)
y_samp=pd.DataFrame(data=y_sample, columns=['Churn'])
df2=pd.concat([X_samp,y_samp], axis=1)

In [16]:
print(df2['Churn'].value_counts())

X=X_samp[train_cols]
y=y_samp["Churn"]

0    483
1    483
Name: Churn, dtype: int64


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=10)

In [19]:
# 스케일링
mean_on_train=X_train.mean(axis=0)
std_on_train=X_train.std(axis=0)

X_train_scaled=(X_train-mean_on_train)/std_on_train
X_test_scaled=(X_test-mean_on_train)/std_on_train

print(X_train_scaled.describe())
print(X_test_scaled.describe())

         Int'l Plan    VMail Plan  VMail Message      Day Mins     Day Calls  \
count  6.760000e+02  6.760000e+02   6.760000e+02  6.760000e+02  6.760000e+02   
mean   1.607853e-16 -1.527378e-16   1.668619e-16 -7.994920e-16  2.461049e-16   
std    1.000000e+00  1.000000e+00   1.000000e+00  1.000000e+00  1.000000e+00   
min   -4.571572e-01 -5.518769e-01  -5.335870e-01 -3.121540e+00 -4.836641e+00   
25%   -4.571572e-01 -5.518769e-01  -5.335870e-01 -6.292776e-01 -6.314072e-01   
50%   -4.571572e-01 -5.518769e-01  -5.335870e-01 -5.332612e-02  3.760733e-02   
75%   -4.571572e-01 -5.518769e-01  -5.335870e-01  7.249207e-01  7.066219e-01   
max    2.184196e+00  1.809318e+00   3.037748e+00  2.589945e+00  3.048173e+00   

         Day Charge      Eve Mins     Eve Calls    Eve Charge    Night Mins  \
count  6.760000e+02  6.760000e+02  6.760000e+02  6.760000e+02  6.760000e+02   
mean  -1.533988e-15 -1.216072e-15  1.018252e-16  3.295769e-15 -7.035792e-16   
std    1.000000e+00  1.000000e+00  1.00000

In [29]:
from sklearn.neural_network import MLPClassifier

model=MLPClassifier(random_state=42, max_iter=2000, hidden_layer_sizes=[100,100])
model.fit(X_train_scaled, y_train)

MLPClassifier(hidden_layer_sizes=[100, 100], max_iter=2000, random_state=42)

In [30]:
print("학습용 :", model.score(X_train_scaled, y_train))
print("검증용 :", model.score(X_test_scaled, y_test))

학습용 : 1.0
검증용 : 0.8344827586206897


In [31]:
# 혼동 행렬
from sklearn.metrics import confusion_matrix

pred=model.predict(X_test_scaled)
cm=confusion_matrix(y_test, pred)
cm

array([[127,  24],
       [ 24, 115]], dtype=int64)

In [33]:
import seaborn as sns

sns.set(font_scale=1.4)
plt.figure(figsize=(9,6))
plt.title("Confusion Matrix")
sns.heatmap(cm, annot=True, fmt="d")

<AxesSubplot:title={'center':'Confusion Matrix'}>